In [1]:
import sys
sys.path.insert(0,'/home/csunix/schtmt/NewFolder/caffe_Sep_anaconda/python')
import numpy as np
import matplotlib.pyplot as plt
import caffe
import h5py
import cv2
import os
import scipy.io
import scipy.misc

caffe.set_device(0)
caffe.set_mode_gpu()

# train on Adam entrance
# net = caffe.Net('test2.prototxt',
#                 '../models/exit_aug1.caffemodel', 
#                 caffe.TEST) 

#aug2
net = caffe.Net('test2.prototxt',
                '../models/exit_aug2.caffemodel', 
                caffe.TEST) 
# print([(k, v.data.shape) for k, v in net.blobs.items()]) # Python data type: list

In [2]:
import fnmatch
import os
from PIL import Image
frame_path='/usr/not-backed-up/1_DATABASE/Adam dataset/Exit/TestSeq/'

# aug1
# save_path = '/usr/not-backed-up/1_convlstm/convLSTM_prediction6_exit/'
# save_path = '/usr/not-backed-up/1_convlstm/convLSTM_prediction6_exit/RemoveTimeStamp'

# aug2
save_path = '/usr/not-backed-up/1_convlstm/prediction6_aug2_exit/RemoveTimeStamp'

if not os.path.exists(save_path):
    os.mkdir(save_path)

for seq in range(4):
    print(seq)    
    # load images                   
    dirpath = (frame_path + 'Test%d.mat' % (seq+1))
    h5file = h5py.File(dirpath,'r')
    vol = h5file['ims'][:]
    vol = np.transpose(vol,[2,1,0])
    nfr = np.shape(vol)[2]
    print(nfr)
    
#     for fr in range(nfr):            
#         if fr < 9:
#             fr_id = '00%d' % (fr+1)                                       
#         elif fr < 99:
#             fr_id = '0%d' % (fr+1)                
#         else:
#             fr_id = '%d' % (fr+1)                
#         im1 = Image.open(dirpath + '/' + fr_id + '.tif')
#         im1 = np.array(im1.resize((227,227),Image.BILINEAR))#size [width,height]            
#         im1 = np.reshape(im1,[1,227,227])
#         im1 = im1.astype(float) / 255.
#         if fr == 0:
#             frame_seq = im1
#         else:
#             frame_seq = np.append(frame_seq,im1,0)
    frame_error = np.zeros([1,nfr-9])
#     print(frame_error.shape)
    for fr in range(nfr-9):
        if fr == 0:
            for idx in range(10):
                im = scipy.misc.imresize(vol[:,:,fr+idx],[227,227],interp='bilinear')
                im = np.reshape(im,[1,227,227])
                im = im.astype(float)/255
                if idx == 0:
                    cur_vol = im
                else:
                    cur_vol = np.append(cur_vol,im,0)
        else:
            cur_vol = cur_vol[1:10,:,:]
            im = scipy.misc.imresize(vol[:,:,fr+9],[227,227],interp='bilinear')
            im = np.reshape(im,[1,227,227])
            im = im.astype(float)/255
            cur_vol = np.append(cur_vol,im,0)                        
        err = []        
        cur_vol_input = cur_vol.reshape([1,10,1,227,227])
        net.blobs['input'].data[...] = cur_vol_input
        out = net.forward()        
        prediction = net.blobs['deconv3'].data
        prediction = np.reshape(prediction,[5,227,227])
#         print(np.shape(cur_vol))
        err = np.power((cur_vol[5:10,:,:] - prediction),2)
        
        # Remove Time Stamp noise in Entrance and Exit datasets
        TimeStampMask = np.ones((5,227,227))
        TimeStampMask[:,181:194,81:115] = 0
        TimeStampMask[:,166:194,126:216] = 0
        err = np.multiply(err,TimeStampMask)
        
        # error of 5 frames
        err = np.reshape(err,[5,227*227])
        err = np.sqrt(np.sum(err,1))
        frame_error[:,fr] = np.sum(err)   
    filename = ("test_%d_error.h5" % (seq+1))        
#     np.save(save_path+file_name,frame_error)
    h5f = h5py.File(os.path.join(save_path,filename), 'w')
    h5f.create_dataset('frame_error', data=frame_error)                
    h5f.close()
        

0
15000
1
15000
2
14999
3
12401
